In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# ---- Config ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256
num_workers = 2
best_model_path = "./models/cifar10_best_model"   # path you used in training

# ---- Data ----
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform
)
test_loader = DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

# ---- Model Definition ----
model = torchvision.models.resnet18(weights=None)
# match your training modifications:
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# ---- Load Checkpoint ----
state = torch.load(best_model_path, map_location=device)
model.load_state_dict(state)
model.to(device)
model.eval()

# ---- Evaluation ----
criterion = nn.CrossEntropyLoss()
running_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

test_loss = running_loss / total
test_acc = 100. * correct / total

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.2f}%")


Files already downloaded and verified
Test Loss: 0.6922
Test Accuracy: 81.23%
